In [1]:
import matplotlib
import matplotlib.pyplot as plt
import torch.nn as nn
import torch
import pandas as pd
import numpy as np
import random as r
from sklearn.model_selection import train_test_split

In [22]:
def normalize(x):
    divisor = x.max(0, keepdim=True)[0]
    normalization_key= divisor.tolist()[0]
    x_normed = x / divisor
    return (x_normed, normalization_key)


In [23]:
data= np.load('Phase01.npy', mmap_mode=None, allow_pickle=True, fix_imports=True,encoding='ASCII')
data = torch.from_numpy(data)
print(data[1,3])
data, normalization_key = normalize(data)
valData, trainData  = train_test_split(data, train_size=.2, random_state=3)
trainInput = trainData[ :,0:3]
trainOutput = trainData[ :,3]
trainOutput = trainOutput.reshape(-1, 1)

valInput = valData[ :,0:3]
valOutput = valData[ :,3]
valOutput = valOutput.reshape(-1, 1)




tensor(1785900., dtype=torch.float64)


In [24]:
data.shape

torch.Size([2970, 4])

In [25]:
class ModeI(nn.Module):

    def __init__(self, input_size, hidden1_size, hidden2_size, num_classes):

        super(ModeI, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden1_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden1_size, hidden2_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden2_size, num_classes)
        self.act_output = nn.Sigmoid()

    def forward(self,x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        out = self.act_output(out)
        return out
model = ModeI(3,16,16,1)
model.double()



ModeI(
  (fc1): Linear(in_features=3, out_features=16, bias=True)
  (relu1): ReLU()
  (fc2): Linear(in_features=16, out_features=16, bias=True)
  (relu2): ReLU()
  (fc3): Linear(in_features=16, out_features=1, bias=True)
  (act_output): Sigmoid()
)

In [26]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [27]:
# compute accuracy (no_grad is optional)
def deviation():
    with torch.no_grad():
        y_pred = model(valInput)

    accuracy = ((abs(y_pred-valOutput)).float().mean())
    return f"Deviation {accuracy}"


In [28]:


def train(batch_size,epoch_number):
    for epoch in range(epoch_number):
        for i in range(0, len(trainInput), batch_size):
            Xbatch = trainInput[i:i+batch_size]
            y_pred = model(Xbatch)
            ybatch = trainOutput[i:i+batch_size]
            loss = loss_fn(y_pred, ybatch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f"Finished epoch {epoch}, latest loss{loss}, {deviation()}")

    


In [29]:
train(5,20)
train(3,20)
train(9,20)
train(5,20)



Finished epoch 0, latest loss0.24771282742067086, Deviation 0.05251574143767357
Finished epoch 1, latest loss0.20876556123093098, Deviation 0.02619709074497223
Finished epoch 2, latest loss0.2056991810056154, Deviation 0.024295736104249954
Finished epoch 3, latest loss0.20582747314418212, Deviation 0.021781062707304955
Finished epoch 4, latest loss0.20582779280670846, Deviation 0.019514048472046852
Finished epoch 5, latest loss0.20570446339942902, Deviation 0.017782242968678474
Finished epoch 6, latest loss0.20558540693218974, Deviation 0.01655220240354538
Finished epoch 7, latest loss0.20546687564668975, Deviation 0.01604580320417881
Finished epoch 8, latest loss0.2053140875889452, Deviation 0.0155853983014822
Finished epoch 9, latest loss0.20507461322851744, Deviation 0.01548720058053732
Finished epoch 10, latest loss0.20490343490454976, Deviation 0.015170887112617493
Finished epoch 11, latest loss0.20461673111895956, Deviation 0.015124352648854256
Finished epoch 12, latest loss0.204

In [30]:
def kequals(force,a, t):
    answer = model(torch.DoubleTensor([force/normalization_key[0],a/normalization_key[1],t/normalization_key[2]]))
    return answer*normalization_key[3]

In [31]:
kequals(53000000,.015,.011)

tensor([13931114.8115], dtype=torch.float64, grad_fn=<MulBackward0>)

In [32]:
normalization_key[3]

34738000.0

In [33]:
deviation()

'Deviation 0.014148558489978313'